### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [5]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}"

In [6]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for i in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    page_url = f"{query_url}&page={i}"
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(page_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    # Try and save the reviews to the reviews_list
    try:
        # Parse the JSON response
        docs = reviews["response"]["docs"]
        # loop through the reviews["response"]["docs"] and append each review to the list
        for j in docs:
            reviews_list.append(j)

        # Print the page that was just retrieved
        print(f'Page number {i+1} retrieved')
        # Print the page number that had no results then break from the loop
    except:
        print(f'Page number {i+1} had no results')
        break


Page number 1 retrieved
Page number 2 retrieved
Page number 3 retrieved
Page number 4 retrieved
Page number 5 retrieved
Page number 6 retrieved
Page number 7 retrieved
Page number 8 retrieved
Page number 9 retrieved
Page number 10 retrieved
Page number 11 retrieved
Page number 12 retrieved
Page number 13 retrieved
Page number 14 retrieved
Page number 15 retrieved
Page number 16 retrieved
Page number 17 retrieved
Page number 18 retrieved
Page number 19 retrieved
Page number 20 retrieved


In [7]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [8]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/what...   
2    https://www.nytimes.com/2023/05/04/movies/you-...   
3    https://www.nytimes.com/2023/04/21/movies/a-to...   
4    https://www.nytimes.com/2023/04/20/movies/othe...   
..                                                 ...   
195  https://www.nytimes.com/2017/03/09/movies/the-...   
196  https://www.nytimes.com/2017/03/09/movies/revi...   
197  https://www.nytimes.com/2017/03/02/movies/love...   
198  https://www.nytimes.com/2017/02/16/movies/ever...   
199  https://www.nytimes.com/2017/02/09/movies/kedi...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Two childhood friends navigate cultural differ...  The New York Times   
2    Religion comes between two girls falling in lo...  The New York Times   
3    Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
4    A radiant Virginie Efira stars as a Parisian t...  The New York Times   
..                                                 ...                 ...   
195  This moody romance stars Tatiana Maslany (“Orp...  The New York Times   
196  A nurse travels to the Ottoman Empire on the e...  The New York Times   
197  Josh Kornbluth runs afoul of the Internal Reve...  The New York Times   
198  A messed-up heroine is asked to choose between...  The New York Times   
199  This documentary profiles distinctive street c...  The New York Times   

                                              keywords  \
0    [{'name': 'subject', 'value': 'Movies', 'rank'...   
1    [{'name': 'subject', 'value': 'Movies', 'rank'...   
2    [{'name': 'subject', 'value': 'Movies', 'rank'...   
3    [{'name': 'subject', 'value': 'Movies', 'rank'...   
4    [{'name': 'subject', 'value': 'Movies', 'rank'...   
..                                                 ...   
195  [{'name': 'subject', 'value': 'Movies', 'rank'...   
196  [{'name': 'subject', 'value': 'Movies', 'rank'...   
197  [{'name': 'creative_works', 'value': 'Love & T...   
198  [{'name': 'subject', 'value': 'Movies', 'rank'...   
199  [{'name': 'creative_works', 'value': 'Kedi (Mo...   

                     pub_date  word_count  \
0    2023-05-25T11:00:03+0000         295   
1    2023-05-04T17:16:45+0000         287   
2    2023-05-04T11:00:08+0000         294   
3    2023-04-21T07:03:25+0000         276   
4    2023-04-20T15:35:13+0000         801   
..                        ...         ...   
195  2017-03-09T21:54:58+0000         251   
196  2017-03-09T21:53:12+0000         267   
197  2017-03-02T21:44:18+0000         246   
198  2017-02-16T21:45:50+0000         256   
199  2017-02-09T16:21:39+0000         274   

                                         headline.main headline.kicker  \
0          ‘The Attachment Diaries’ Review: Love, Sick            None   
1    Review: ‘What’s Love Got to Do With It?’ Proba...            None   
2    ‘You Can Live Forever’ Review: Do You Love Me ...            None   
3    ‘A Tourist’s Guide to Love’ Review: A Wearying...            None   
4       ‘Other People’s Children’ Review: True Romance   Critic’s pick   
..                                                 ...             ...   
195  Review: A Combustible Pair Find Love in ‘The O...            None   
196  Review: Love as the World Wars, in ‘The Ottoma...            None   
197  Review: It’s All Mirth and Taxes in ‘Love & Ta...            None   
198  Review: ‘Everybody Loves Somebody,’ a Rom-Com ...            None   
199  Review: Cute Cats of ‘Kedi,’ Rekindling a ‘Lov...            None   

    headline.content_kicker  \
0                      None   
1                      None   
2                      None   
3                      None   
4                      None   
..                      ...   
195                    None   
196                    None   
197    

In [9]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

In [10]:
reviews_df

web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/what...   
2    https://www.nytimes.com/2023/05/04/movies/you-...   
3    https://www.nytimes.com/2023/04/21/movies/a-to...   
4    https://www.nytimes.com/2023/04/20/movies/othe...   
..                                                 ...   
195  https://www.nytimes.com/2017/03/09/movies/the-...   
196  https://www.nytimes.com/2017/03/09/movies/revi...   
197  https://www.nytimes.com/2017/03/02/movies/love...   
198  https://www.nytimes.com/2017/02/16/movies/ever...   
199  https://www.nytimes.com/2017/02/09/movies/kedi...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Two childhood friends navigate cultural differ...  The New York Times   
2    Religion comes between two girls falling in lo...  The New York Times   
3    Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
4    A radiant Virginie Efira stars as a Parisian t...  The New York Times   
..                                                 ...                 ...   
195  This moody romance stars Tatiana Maslany (“Orp...  The New York Times   
196  A nurse travels to the Ottoman Empire on the e...  The New York Times   
197  Josh Kornbluth runs afoul of the Internal Reve...  The New York Times   
198  A messed-up heroine is asked to choose between...  The New York Times   
199  This documentary profiles distinctive street c...  The New York Times   

                                              keywords  \
0    [{'name': 'subject', 'value': 'Movies', 'rank'...   
1    [{'name': 'subject', 'value': 'Movies', 'rank'...   
2    [{'name': 'subject', 'value': 'Movies', 'rank'...   
3    [{'name': 'subject', 'value': 'Movies', 'rank'...   
4    [{'name': 'subject', 'value': 'Movies', 'rank'...   
..                                                 ...   
195  [{'name': 'subject', 'value': 'Movies', 'rank'...   
196  [{'name': 'subject', 'value': 'Movies', 'rank'...   
197  [{'name': 'creative_works', 'value': 'Love & T...   
198  [{'name': 'subject', 'value': 'Movies', 'rank'...   
199  [{'name': 'creative_works', 'value': 'Kedi (Mo...   

                     pub_date  word_count  \
0    2023-05-25T11:00:03+0000         295   
1    2023-05-04T17:16:45+0000         287   
2    2023-05-04T11:00:08+0000         294   
3    2023-04-21T07:03:25+0000         276   
4    2023-04-20T15:35:13+0000         801   
..                        ...         ...   
195  2017-03-09T21:54:58+0000         251   
196  2017-03-09T21:53:12+0000         267   
197  2017-03-02T21:44:18+0000         246   
198  2017-02-16T21:45:50+0000         256   
199  2017-02-09T16:21:39+0000         274   

                                         headline.main headline.kicker  \
0          ‘The Attachment Diaries’ Review: Love, Sick            None   
1    Review: ‘What’s Love Got to Do With It?’ Proba...            None   
2    ‘You Can Live Forever’ Review: Do You Love Me ...            None   
3    ‘A Tourist’s Guide to Love’ Review: A Wearying...            None   
4       ‘Other People’s Children’ Review: True Romance   Critic’s pick   
..                                                 ...             ...   
195  Review: A Combustible Pair Find Love in ‘The O...            None   
196  Review: Love as the World Wars, in ‘The Ottoma...            None   
197  Review: It’s All Mirth and Taxes in ‘Love & Ta...            None   
198  Review: ‘Everybody Loves Somebody,’ a Rom-Com ...            None   
199  Review: Cute Cats of ‘Kedi,’ Rekindling a ‘Lov...            None   

    headline.content_kicker  \
0                      None   
1                      None   
2                      None   
3                      None   
4                      None   
..                      ...   
195                    None   
196                    None   
197    

In [11]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df['keywords']

0      subject: Movies;creative_works: The Attachment...
1      subject: Movies;persons: Kapur, Shekhar;person...
2      subject: Movies;creative_works: You Can Live F...
3      subject: Movies;creative_works: A Tourist's Gu...
4      subject: Movies;persons: Zlotowski, Rebecca;cr...
                             ...                        
195    subject: Movies;creative_works: The Other Half...
196    subject: Movies;creative_works: The Ottoman Li...
197    creative_works: Love & Taxes (Movie);persons: ...
198    subject: Movies;creative_works: Everybody Love...
199    creative_works: Kedi (Movie);subject: Document...
Name: keywords, Length: 200, dtype: object

In [12]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_query = reviews_df['title'].to_list()
title_query

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [14]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [15]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for i in title_query:
    # Check if we need to sleep before making a request
    if request_counter == 50:
        time.sleep(1)
        request_counter = 1
        print('Application is spleeping')

    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    page_url = url+i+tmdb_key_string
    movies = requests.get(page_url).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = movies['results'][0]['id']
        movie_url = f'https://api.themoviedb.org/3/movie/{movie_id}?{tmdb_key_string}' 
        
        # Make a request for a the full movie details
        # Execute "GET" request with url
        movies = requests.get(movie_url).json()
        
        # Extract the genre names into a list
        genres = movies['genres']
        genre_names = [genre['name'] for genre in genres]
        
        # Extract the spoken_languages' English name into a list
        languages = movies['spoken_languages']
        spoken_languages = [language['english_name'] for language in languages]
        
        # Extract the production_countries' name into a list
        countries = movies['production_countries']
        production_countries = [country['name'] for country in countries]
        
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_info = {'title': movies['title'], 
        'original_title': movies['original_title'], 
        'budget': movies['budget'], 
        'original_language': movies['original_language'], 
        'homepage': movies['homepage'],
        'overview': movies['overview'], 
        'popularity': movies['popularity'],
        'runtime': movies['runtime'],
        'revenue': movies['revenue'],
        'release_date': movies['release_date'],
        'vote_average': movies['vote_average'],
        'vote_count': movies['vote_count'], 
        'genres': genre_names,
        'spoken_languages': spoken_languages,
        'production_countries': production_countries}
        
        tmdb_movies_list.append(movie_info)
        
        # Print out the title that was found
        print('Found title: ' + movies['title'])
    except:
        print('Title not found')


Found title: The Attachment Diaries
Title not found
Found title: You Can Live Forever
Found title: A Tourist's Guide to Love
Found title: Other People's Children
Found title: One True Loves
Found title: The Lost Weekend: A Love Story
Found title: A Thousand and One
Found title: Your Place or Mine
Found title: Love in the Time of Fentanyl
Found title: Pamela, A Love Story
Found title: In from the Side
Found title: After Love
Found title: Alcarràs
Found title: Nelly and Monsieur Arnaud
Found title: Lady Chatterley's Lover
Found title: The Sound of Christmas
Found title: The Inspection
Found title: Bones and All
Found title: My Policeman
Found title: About Fate
Found title: Waiting for Bojangles
Found title: I Love My Dad
Found title: A Love Song
Found title: Alone Together
Found title: Art of Love
Found title: 2099: The Soldier Protocol
Found title: Thor: Love and Thunder
Found title: Both Sides of the Blade
Found title: Fire of Love
Found title: Art of Love
Found title: Stay Prayed Up
F

In [16]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 4.08,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
  

In [17]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

title             original_title    budget  \
0       The Attachment Diaries                   El apego         0   
1         You Can Live Forever       You Can Live Forever         0   
2    A Tourist's Guide to Love  A Tourist's Guide to Love         0   
3      Other People's Children     Les Enfants des autres         0   
4               One True Loves             One True Loves         0   
..                         ...                        ...       ...   
149              The Last Face              The Last Face         0   
150              Lost in Paris            Paris pieds nus         0   
151             The Other Half             The Other Half         0   
152     The Ottoman Lieutenant     The Ottoman Lieutenant  40000000   
153                Art of Love            Romantik Hırsız         0   

    original_language                                           homepage  \
0                  es                                                      
1                  en  https://gooddeedentertainment.com/you-can-live...   
2                  en             https://www.netflix.com/title/81424906   
3                  fr  https://www.wildbunch.biz/movie/other-peoples-...   
4                  en                                                      
..                ...                                                ...   
149                en                                                      
150                fr                                                      
151                en                      http://theotherhalfmovie.com/   
152                en  https://www.uphe.com/movies/the-ottoman-lieute...   
153                tr             https://www.netflix.com/title/81689799   

                                              overview  popularity  runtime  \
0    Argentina, 1970s. A desperate young woman goes...       4.080      102   
1    When Jaime, a gay teenager, is sent to live in...      21.650       96   
2    After an unexpected break up, a travel executi...      23.807       96   
3    Rachel loves her life, her students, her frien...      15.227      104   
4    Emma and Jesse are living the perfect life tog...      16.082      100   
..                                                 ...         ...      ...   
149  Miguel, a heroic Spanish doctor, puts himself ...      16.571      130   
150  Fiona visits Paris for the first time to assis...       9.965       83   
151  A grief-stricken man and a bipolar woman fall ...       5.240      103   
152  Lillie, a determined American woman, ventures ...      14.898      111   
153  After learning that the art thief she has been...      77.201       99   

     revenue release_date  vote_average  vote_count  \
0          0   2021-10-07         3.000           4   
1      15055   2023-03-24         6.500          38   
2          0   2023-04-21         6.311         167   
3      84178   2022-09-21         6.814         191   
4      37820   2023-04-07         6.513          77   
..       ...          ...           ...         ...   
149        0   2017-01-11         5.549         194   
150        0   2017-01-14         6.022         114   
151        0   2016-12-02         6.300          24   
152   413844   2017-03-10         6.164         247   
153        0   2024-03-13         6.457         115   

                                 genres       spoken_languages  \
0    [Drama, Mystery, Thriller, Horror]              [Spanish]   
1                      [Drama, Romance]      [English, French]   
2                     [Romance, Comedy]  [English, Vietnamese]   
3                       [Drama, Comedy]      [French, English]   
4              [Romance, Comedy, Drama]     [English, Spanish]   
..                                  ...                    ...   
149                             [Drama]  [English, Portuguese]   
150                            [Comedy]               [French]   
151                    [Drama, Romance]              [E

### Merge and Clean the Data for Export

In [19]:
# Merge the New York Times reviews and TMDB DataFrames on title
tmdb_nyt_df = pd.merge(tmdb_df, reviews_df, on='title', how='inner')
tmdb_nyt_df

title                  original_title    budget  \
0            The Attachment Diaries                        El apego         0   
1              You Can Live Forever            You Can Live Forever         0   
2                    One True Loves                  One True Loves         0   
3    The Lost Weekend: A Love Story  The Lost Weekend: A Love Story         0   
4                A Thousand and One              A Thousand and One         0   
..                              ...                             ...       ...   
121                   The Last Face                   The Last Face         0   
122                   Lost in Paris                 Paris pieds nus         0   
123                  The Other Half                  The Other Half         0   
124          The Ottoman Lieutenant          The Ottoman Lieutenant  40000000   
125                     Art of Love                 Romantik Hırsız         0   

    original_language                                           homepage  \
0                  es                                                      
1                  en  https://gooddeedentertainment.com/you-can-live...   
2                  en                                                      
3                  en                   https://thelostweekendmovie.com/   
4                  en  https://www.focusfeatures.com/a-thousand-and-one/   
..                ...                                                ...   
121                en                                                      
122                fr                                                      
123                en                      http://theotherhalfmovie.com/   
124                en  https://www.uphe.com/movies/the-ottoman-lieute...   
125                tr             https://www.netflix.com/title/81689799   

                                              overview  popularity  runtime  \
0    Argentina, 1970s. A desperate young woman goes...       4.080      102   
1    When Jaime, a gay teenager, is sent to live in...      21.650       96   
2    Emma and Jesse are living the perfect life tog...      16.082      100   
3    May Pang lovingly recounts her life in rock & ...      27.253       95   
4    Struggling but unapologetically living on her ...      12.845      116   
..                                                 ...         ...      ...   
121  Miguel, a heroic Spanish doctor, puts himself ...      16.571      130   
122  Fiona visits Paris for the first time to assis...       9.965       83   
123  A grief-stricken man and a bipolar woman fall ...       5.240      103   
124  Lillie, a determined American woman, ventures ...      14.898      111   
125  After learning that the art thief she has been...      77.201       99   

     revenue release_date  ...  \
0          0   2021-10-07  ...   
1      15055   2023-03-24  ...   
2      37820   2023-04-07  ...   
3      77145   2023-04-13  ...   
4    3395595   2023-03-31  ...   
..       ...          ...  ...   
121        0   2017-01-11  ...   
122        0   2017-01-14  ...   
123        0   2016-12-02  ...   
124   413844   2017-03-10  ...   
125        0   2024-03-13  ...   

                                         headline.main  headline.kicker  \
0          ‘The Attachment Diaries’ Review: Love, Sick             None   
1    ‘You Can Live Forever’ Review: Do You Love Me ...             None   
2       ‘One True Loves’ Review: A Romance Lost at Sea             None   
3    ‘The Lost Weekend: A Love Story’ Review: When ...             None   
4    ‘A Thousand and One’ Review: A New York Love S...    Critic’s Pick   
..                                                 ...              ...   
121  Review: Aid Workers in Love and War in Sean Pe...             None   
122  Review: Finding Love (and Slapstick) While ‘Lo...             None   
123  Review: A Combustible Pair Find Love in ‘The O...             None   
124  Review: Love as the World Wars,

In [20]:
tmdb_nyt_df['genres']

0      [Drama, Mystery, Thriller, Horror]
1                        [Drama, Romance]
2                [Romance, Comedy, Drama]
3                           [Documentary]
4                          [Drama, Crime]
                      ...                
121                               [Drama]
122                              [Comedy]
123                      [Drama, Romance]
124                 [Romance, Drama, War]
125              [Romance, Action, Drama]
Name: genres, Length: 126, dtype: object

In [21]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]

def remove_char(row, character):
    for char in character:
        row = row.replace(char, '')
    return row
# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    tmdb_nyt_df[col] = tmdb_nyt_df[col].astype(str)
    
    # Loop through characters to remove
    for char in characters_to_remove:
        tmdb_nyt_df[col] = tmdb_nyt_df[col].apply(lambda text: remove_char(text, char))

# Display the fixed DataFrame
# Was having trouble displaying all columns so I am using this command
pd.set_option('display.max_columns', None)

tmdb_nyt_df.head(20)

title                  original_title     budget  \
0           The Attachment Diaries                        El apego          0   
1             You Can Live Forever            You Can Live Forever          0   
2                   One True Loves                  One True Loves          0   
3   The Lost Weekend: A Love Story  The Lost Weekend: A Love Story          0   
4               A Thousand and One              A Thousand and One          0   
5               Your Place or Mine              Your Place or Mine          0   
6     Love in the Time of Fentanyl    Love in the Time of Fentanyl          0   
7                       After Love                      After Love          0   
8                         Alcarràs                        Alcarràs          3   
9           The Sound of Christmas          The Sound of Christmas          0   
10                  The Inspection                  The Inspection          0   
11                   Bones and All                   Bones and All   16000000   
12                    My Policeman                    My Policeman          0   
13                      About Fate                      About Fate          0   
14           Waiting for Bojangles          En attendant Bojangles          0   
15                   I Love My Dad                   I Love My Dad          0   
16                     A Love Song                     A Love Song          0   
17                  Alone Together                  Alone Together          0   
18                     Art of Love                 Romantik Hırsız          0   
19          Thor: Love and Thunder          Thor: Love and Thunder  250000000   

   original_language                                           homepage  \
0                 es                                                      
1                 en  https://gooddeedentertainment.com/you-can-live...   
2                 en                                                      
3                 en                   https://thelostweekendmovie.com/   
4                 en  https://www.focusfeatures.com/a-thousand-and-one/   
5                 en             https://www.netflix.com/title/81045831   
6                 en                                                      
7                 en                                                      
8                 ca                     https://www.alcarras-film.com/   
9                 en                                                      
10                en              https://a24films.com/films/inspection   
11                it                   https://www.bonesandallfilm.net/   
12                en               https://www.amazon.com/dp/B09Y8SHDPD   
13                en                                                      
14                fr                                                      
15                en             http://www.magpictures.com/ilovemydad/   
16                en                                                      
17                en                                                      
18                tr             https://www.netflix.com/title/81689799   
19                en  https://www.marvel.com/movies/thor-love-and-th...   

                                             overview  popularity  runtime  \
0   Argentina, 1970s. A desperate young woman goes...       4.080      102   
1   When Jaime, a gay teenager, is sent to live in...      21.650       96   
2   Emma and Jesse are living the perfect life tog...      16.082      100   
3   May Pang lovingly recounts her life in rock & ...      27.253       95   
4   Struggling but unapologetically living on her ...      12.845      116   
5   When best friends and total opposites Debbie a...      23.759      109   
6   An intimate portrait of a community fighting t...       1.777       85   
7   Set in the port town of Dover in the South-Eas...      17.188       89   
8   In a small village in Catalonia, the peach fa

In [22]:
# Drop "byline.person" column
tmdb_nyt_df = tmdb_nyt_df.drop('byline.person', axis=1)

In [23]:
tmdb_nyt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    126 non-null    object 
 1   original_title           126 non-null    object 
 2   budget                   126 non-null    int64  
 3   original_language        126 non-null    object 
 4   homepage                 126 non-null    object 
 5   overview                 126 non-null    object 
 6   popularity               126 non-null    float64
 7   runtime                  126 non-null    int64  
 8   revenue                  126 non-null    int64  
 9   release_date             126 non-null    object 
 10  vote_average             126 non-null    float64
 11  vote_count               126 non-null    int64  
 12  genres                   126 non-null    object 
 13  spoken_languages         126 non-null    object 
 14  production_countries     1

In [24]:
# Delete duplicate rows and reset index
tmdb_nyt_df = tmdb_nyt_df.drop_duplicates()

In [25]:
tmdb_nyt_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 123 entries, 0 to 124
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    123 non-null    object 
 1   original_title           123 non-null    object 
 2   budget                   123 non-null    int64  
 3   original_language        123 non-null    object 
 4   homepage                 123 non-null    object 
 5   overview                 123 non-null    object 
 6   popularity               123 non-null    float64
 7   runtime                  123 non-null    int64  
 8   revenue                  123 non-null    int64  
 9   release_date             123 non-null    object 
 10  vote_average             123 non-null    float64
 11  vote_count               123 non-null    int64  
 12  genres                   123 non-null    object 
 13  spoken_languages         123 non-null    object 
 14  production_countries     123 no

In [26]:
# Export data to CSV without the index
tmdb_nyt_df.to_csv('movie_data.csv', index=False)